In [1]:
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer
import numpy as np
from tqdm import tqdm
# Load the model and tokenizer
model_id = "google/gemma-2-9b-it"
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Make sure pad_token_id is set properly
pad_token_id = tokenizer.eos_token_id
pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

/home/admin/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:07<00:00,  1.79s/it]


In [2]:
system_prompt = "You are a highly accurate question-answering assistant. Answer concisely with only the best option (A, B, C, or D), formatted as 'Answer: X'. Do not add any other text to your answer, only stick to the options."

def single_inference_llama(question, system_prompt=system_prompt):
    # Tokenize the input question
    tokenizer.add_special_tokens({'pad_token': pad_token})
    inputs = tokenizer(system_prompt + "\n\n" + question, return_tensors="pt", padding=True).to(model.device)
    

    # Generate response with limited tokens (for short answers)
    with torch.no_grad():
        outputs = model.generate(
            inputs.input_ids,
            attention_mask=inputs.attention_mask,
            max_new_tokens=5,  # Limit to few tokens
            pad_token_id=pad_token_id,
            return_dict_in_generate=True,
            output_scores=True,
            temperature=1
        )

    # Extract only the generated tokens (after input tokens) as a list and decode each one
    generated_tokens_list = outputs.sequences[0][len(inputs.input_ids[0]):]
    decoded_tokens_list = [tokenizer.decode([token]) for token in generated_tokens_list]
    # print("Decoded tokens:", decoded_tokens_list)

    # Extract logits for the generated tokens
    logits = outputs.scores

    # Calculate log probabilities for each token in the generated response
    log_probs = []
    linear_probs = []
    for i, logits_per_token in enumerate(logits):
        probs = torch.nn.functional.log_softmax(logits_per_token, dim=-1)
        token_id = generated_tokens_list[i]
        log_prob = probs[0, token_id].item()  # Get log probability of the token
        log_probs.append(log_prob)
        
        # Convert log probability to linear probability (percentage)
        linear_prob = np.exp(log_prob) * 100
        linear_probs.append(linear_prob)

    # Print logprobs and linear probabilities for each token
    # print("\nLogprobs and Linear Probabilities for each generated token:")
    # for token, log_prob, linear_prob in zip(generated_tokens_list, log_probs, linear_probs):
    #     token_str = tokenizer.decode([token])
    #     print(f"Token: {token_str}, Logprob: {log_prob}, Linear Probability: {linear_prob:.2f}%")

    return decoded_tokens_list, log_probs, linear_probs




In [3]:

import pandas as pd

df = pd.read_csv('/scratch/essa/Confidence_LLM/llm-confidence-study-main/MMLU_5000_sample.csv')[:1000]

responses = []

for i, row in tqdm(df.iterrows()):
    question = row['prompt']
    answer = row['answer']

    response = single_inference_llama(question)

    responses.append(response)


new_df = pd.DataFrame(responses, columns=['generated', 'log_probs', 'linear_probs'])
new_df['correct_answer'] = df['answer']

new_df.to_csv("gemma_lobprob_benchmark_mmlu.csv", index=False)

1000it [11:36,  1.44it/s]


In [ ]:

import pandas as pd

df = pd.read_csv('/scratch/essa/Confidence_LLM/llm-confidence-study-main/MedMCQA_5000_sample.csv')[:1000]

responses = []

for i, row in tqdm(df.iterrows()):
    question = row['prompt']
    answer = row['answer']

    response = single_inference_llama(question)

    responses.append(response)


new_df = pd.DataFrame(responses, columns=['generated', 'log_probs', 'linear_probs'])
new_df['correct_answer'] = df['answer']

new_df.to_csv("gemma_lobprob_benchmark_medmcqa.csv", index=False)

1000it [11:27,  1.46it/s]


In [4]:

import pandas as pd

df = pd.read_csv('/scratch/essa/Confidence_LLM/llm-confidence-study-main/ScienceMCQ_5000_sample.csv')[:1000]

responses = []

for i, row in tqdm(df.iterrows()):
    question = row['prompt']
    answer = row['answer']

    response = single_inference_llama(question)

    responses.append(response)


new_df = pd.DataFrame(responses, columns=['generated', 'log_probs', 'linear_probs'])
new_df['correct_answer'] = df['answer']

new_df.to_csv("gemma_lobprob_benchmark_sciencemcq.csv", index=False)

1000it [14:17,  1.17it/s]


### Shufflled Data

In [5]:

import pandas as pd

df = pd.read_csv('/scratch/essa/Confidence_LLM/llm-confidence-study-main/Shuffled_Datasets/SHUFFLED_MedMCQA_5000_sample.csv')[:1000]

responses = []

for i, row in tqdm(df.iterrows()):
    question = row['prompt']
    answer = row['answer']

    response = single_inference_llama(question)

    responses.append(response)


new_df = pd.DataFrame(responses, columns=['generated', 'log_probs', 'linear_probs'])
new_df['correct_answer'] = df['answer']

new_df.to_csv("gemmaa_lobprob_benchmark_medmcqa_shuffled.csv", index=False)

1000it [11:27,  1.45it/s]


In [6]:

import pandas as pd

df = pd.read_csv('/scratch/essa/Confidence_LLM/llm-confidence-study-main/Shuffled_Datasets/SHUFFLED_MMLU_5000_sample.csv')[:1000]

responses = []

for i, row in tqdm(df.iterrows()):
    question = row['prompt']
    answer = row['answer']

    response = single_inference_llama(question)

    responses.append(response)


new_df = pd.DataFrame(responses, columns=['generated', 'log_probs', 'linear_probs'])
new_df['correct_answer'] = df['answer']

new_df.to_csv("gemma_lobprob_benchmark_mmlu_shuffled.csv", index=False)

0it [00:00, ?it/s]

1000it [12:01,  1.39it/s]


In [7]:

import pandas as pd

df = pd.read_csv('/scratch/essa/Confidence_LLM/llm-confidence-study-main/Shuffled_Datasets/SHUFFLED_ScienceMCQ_5000_sample.csv')[:1000]

responses = []

for i, row in tqdm(df.iterrows()):
    question = row['prompt']
    answer = row['answer']

    response = single_inference_llama(question)

    responses.append(response)


new_df = pd.DataFrame(responses, columns=['generated', 'log_probs', 'linear_probs'])
new_df['correct_answer'] = df['answer']

new_df.to_csv("gemma_lobprob_benchmark_sciencemcq_shuffled.csv", index=False)

1000it [14:29,  1.15it/s]
